**Topic Modelling**

In [23]:
import os
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
import nltk
nltk.download('wordnet')
import gensim
from gensim import corpora
import pandas as pd

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/andreaprenner/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
!python3 -m pip install --upgrade pandas

     |████████████████████████████████| 10.6MB 95kB/s eta 0:00:013


In [21]:
#Set WD
os.chdir("/Users/andreaprenner/Desktop/Master_UvA/1. Semester/FundamentalsOfDS/Assignment1")

In [22]:
data_formatted = pd.read_pickle("data_formatted.pickle")
data_formatted

ModuleNotFoundError: No module named 'pandas.core.internals.managers'; 'pandas.core.internals' is not a package

In [2]:
data_sentimentanalysis = data_formatted.copy()

NameError: name 'data_formatted' is not defined

In [3]:
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(w) for w in text]

In [4]:
stopwords = stopwords.words('english')

In [ ]:
#Data cleaning - removing punctuation from text; still need to remove hashes
data_sentimentanalysis['text'] = data_sentimentanalysis['text'].str.replace('@[^\s]+','')
data_sentimentanalysis['text'] = data_sentimentanalysis['text'].str.replace('[^\w\s]','')
data_sentimentanalysis['text'] = data_sentimentanalysis['text'].str.replace('\[.*?\]','')
data_sentimentanalysis['text'] = data_sentimentanalysis['text'].str.replace('[‘’“”…]','')
data_sentimentanalysis['text'] = data_sentimentanalysis['text'].str.replace('\w*\d\w*','')
data_sentimentanalysis['text'] = data_sentimentanalysis['text'].apply(lambda x: x.lower())
data_sentimentanalysis['text'] = data_sentimentanalysis['text'].apply(lambda x: x.split())
data_sentimentanalysis['text'] = data_sentimentanalysis['text'].apply(lambda x: [item for item in x if item not in stopwords])
data_sentimentanalysis['text'] = data_sentimentanalysis['text'].apply(lemmatize_text)

In [ ]:
#Create data set for trump vs clinton
trump_mentioned_true = data_sentimentanalysis[data_sentimentanalysis['Trump']]
clinton_mentioned_true = data_sentimentanalysis[data_sentimentanalysis['Clinton']]

In [ ]:
#Filter out rows where text doesnt contain any entries after removing tags and hashes
trump_mentioned_true = trump_mentioned_true[trump_mentioned_true['text'].map(lambda d: len(d)) > 0]
trump_text = trump_mentioned_true.text.tolist()
dictionary_trump = corpora.Dictionary(trump_text)
doc_term_matrix_trump = [dictionary_trump.doc2bow(doc) for doc in trump_text]
# Creating the object for LDA model using gensim library
Lda_trump = gensim.models.ldamodel.LdaModel
# Running and Trainign LDA model on the document term matrix.
ldamodel_trump = Lda_trump(doc_term_matrix_trump, num_topics=2, id2word = dictionary_trump, passes=100)
topics_trump = ldamodel_trump.print_topics(num_topics=2, num_words=8)
topics_trump

In [ ]:
#Filter out rows where text doesnt contain any entries after removing tags and hashes
clinton_mentioned_true = clinton_mentioned_true[clinton_mentioned_true['text'].map(lambda d: len(d)) > 0]
clinton_text = clinton_mentioned_true.text.tolist()
dictionary_clinton = corpora.Dictionary(clinton_text)
doc_term_matrix_clinton = [dictionary_clinton.doc2bow(doc) for doc in clinton_text]
# Creating the object for LDA model using gensim library
Lda_clinton = gensim.models.ldamodel.LdaModel
# Running and Trainign LDA model on the document term matrix.
ldamodel_clinton = Lda_clinton(doc_term_matrix_clinton, num_topics=2, id2word = dictionary_clinton, passes=100)
topics_clinton = ldamodel_clinton.print_topics(num_topics=10, num_words=4)
topics_clinton

**Just using nouns in Topic Modelling**

In [5]:
# Let's create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag
import numpy as np

In [ ]:
#Apply function that determines the type of each word
trump_mentioned_text = trump_mentioned_true['text'].apply(lambda x: nltk.pos_tag(x))

In [ ]:
#Filter out the nouns for each row
trump_mentioned_nouns = pd.DataFrame(np.zeros((len(trump_mentioned_true), 1)))
trump_mentioned_nouns.columns = ['Text']
for i in range(0, len(trump_mentioned_text)):
    nouns = [word for (word, pos) in trump_mentioned_text.iloc[i] if is_noun(pos)]
    nouns = ' '.join(nouns)
    trump_mentioned_nouns.iloc[i] = nouns

In [ ]:
#Split the Text 
trump_mentioned_nouns['Text'] = trump_mentioned_nouns['Text'].apply(lambda x: x.split())

In [ ]:
trump_text = trump_mentioned_nouns.Text.tolist()
dictionary_trump = corpora.Dictionary(trump_text)
doc_term_matrix_trump = [dictionary_trump.doc2bow(doc) for doc in trump_text]
Lda_trump = gensim.models.ldamodel.LdaModel
ldamodel_trump = gensim.models.LdaMulticore(doc_term_matrix_trump, num_topics=10, id2word=dictionary_trump, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

In [ ]:
#Test specific document how it would have been classified 
bow_doc_1 = doc_term_matrix_trump[5]

In [ ]:
for i in range(len(bow_doc_1)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_1[i][0], 
                                               dictionary_trump[bow_doc_1[i][0]], 
bow_doc_1[i][1]))

In [ ]:
for index, score in sorted(ldamodel_trump[bow_doc_1], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, ldamodel_trump.print_topic(index, 10)))

In [ ]:
#Apply function that determines the type of each word
clinton_mentioned_text = trump_mentioned_true['text'].apply(lambda x: nltk.pos_tag(x))

In [6]:
#Filter out the nouns for each row
clinton_mentioned_nouns = pd.DataFrame(np.zeros((len(clinton_mentioned_true), 1)))
clinton_mentioned_nouns.columns = ['Text']
for i in range(0, len(clinton_mentioned_text)):
    nouns = [word for (word, pos) in clinton_mentioned_nouns.iloc[i] if is_noun(pos)]
    nouns = ' '.join(nouns)
    clinton_mentioned_nouns.iloc[i] = nouns

NameError: name 'pd' is not defined

In [ ]:
#Split the text 
clinton_mentioned_nouns['Text'] = clinton_mentioned_nouns['Text'].apply(lambda x: x.split())

In [ ]:
clinton_text = clinton_mentioned_nouns.text.tolist()
dictionary_clinton = corpora.Dictionary(clinton_text)
doc_term_matrix_clinton = [dictionary_clinton.doc2bow(doc) for doc in clinton_text]
Lda_clinton = gensim.models.ldamodel.LdaModel
ldamodel_clinton = Lda_clinton(doc_term_matrix_clinton, num_topics=10, id2word = dictionary_clinton, passes=100)
topics_clinton = ldamodel_clinton.print_topics(num_topics=10, num_words=8)
topics_clinton